# Retrieval

LangChain에서 Retrieval은 외부 데이터에서 관련 정보를 찾아 프롬프트에 포함시켜(Context) LLM에 전달하는 역할을 한다. 주요 구성 요소는 다음과 같다.

- **Document Loader**: 다양한 원본 데이터를 LangChain 표준 문서 객체로 변환한다.
- **Text Splitter**: 긴 문서를 작은 청크로 분할해 검색 효율을 높인다.
- **Embedding Model**: 텍스트를 의미 기반 벡터로 변환한다.
- **Vector Store**: 임베딩된 벡터를 저장하고 유사도 기반 검색을 지원한다.
- **Retriever**: 쿼리에 대해 관련 문서를 찾아주는 표준 인터페이스를 제공한다.
- **Retrieval Chain**: 검색된 문서를 LLM에 전달해 답변을 생성하는 체인 구조를 제공한다.

이렇게 각 모듈이 결합되어, 외부 데이터 기반의 효과적인 검색 및 답변 생성이 가능하다.

**환각 Hallucination:**

LLM이 실제 근거 없이 그럴듯해 보이는 정보를 생성하는 현상이다.

**주요 원인**
1. **학습 데이터 한계**
   * 모델이 학습한 데이터에 해당 정보가 없거나 부족할 때 발생한다.
2. **확률적 생성 과정**
   * 토큰 예측 시 언어적 일관성을 우선하다 보니, 사실 여부가 검증되지 않은 내용을 생성한다.
3. **프롬프트 모호성**
   * 지시가 불명확하거나 맥락이 부족하면 모델이 관련 없는 정보를 보충·왜곡한다.

**대표 사례**
* 존재하지 않는 논문·저자명을 인용함.
* 역사적·과학적 사실을 잘못 기술함.
* 실행 불가능하거나 비효율적인 코드 제안.


**완화 방안**

1. **지식 기반 검색 결합**
   * Retrieval-Augmented Generation(RAG) 방식으로 외부 문서·데이터베이스에서 실시간 근거를 가져와 보강한다.
2. **프롬프트 구체화**
   * “출처를 함께 제시해 달라” 등 명시적 요청을 통해 근거 표기를 유도한다.
3. **후처리 검증**
   * 생성 결과를 룰 기반 검증 또는 전문가 리뷰를 통해 교차 확인한다.
4. **모델 파인튜닝 및 앙상블**
   * 도메인 특화 데이터로 추가 학습하거나, 룰 기반 시스템과 결합하여 정확도를 높인다.

In [ ]:
#%pip install langchain langchain-community langchain-openai langchain-huggingface wikipedia pypdf tavily-python tiktoken faiss-cpu sentence-transformers -Uqqq

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv  # .env 파일의 환경변수 로드
import os                       # 환경변수 접근용

load_dotenv()                   # 현재 위치의 .env를 읽어와 환경변수로 등록
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")  # .env의 openai_key 값을 OPENAI_API_KEY로 등록
os.environ["LANGSMITH_TRACING"] = 'true'                # LangSmith 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"] = 'https://api.smith.langchain.com'  # LangSmith API 엔드포인트 설정
os.environ["LANGSMITH_PROJECT"] = 'skn23-langchain'                   # LangSmith 프로젝트명 설정
os.environ["LANGSMITH_API_KEY"] = os.getenv("langsmith_key")          # .env의 langsmith_key 값을 LANGSMITH_API_KEY로 등록

## Document

Document는 LangChain 프레임워크에서 다양한 데이터 소스(예: 텍스트 파일, PDF, 웹페이지 등)로부터 불러온 정보를 표준화된 객체로 표현하는 핵심 데이터 구조이다. 이 객체는 언어 모델(LLM)이 외부 데이터를 이해하고 처리할 수 있도록 도와준다.

**Document 객체의 구조**
1. page_content: 문서의 실제 내용을 담고 있는 문자열(str)이다. 예를 들어, 텍스트 파일의 본문이나 PDF의 텍스트 등이 여기에 저장된다.
2. metadata: 문서에 대한 부가 정보를 담는 딕셔너리(dict) 형태의 속성이다. 예를 들어, 파일 경로, 페이지 번호, 작성자, 데이터 출처 등 다양한 메타데이터를 저장할 수 있다.


**Document의 역할과 활용**
1. 표준화된 데이터 구조: 다양한 포맷의 데이터를 일관된 방식으로 표현하여, LLM이 손쉽게 접근하고 활용할 수 있도록 한다.
2. 문서 처리의 기본 단위: LangChain의 문서 로더(Document Loader)는 파일, 웹, 데이터베이스 등 여러 소스에서 데이터를 읽어와 Document 객체로 변환한다.
3. 청크 단위 분할: 대용량 문서는 작은 단위(청크)로 쪼개어 각각의 Document로 저장하고, 검색 및 임베딩 처리에 활용한다.

In [3]:
from langchain_core.documents import Document  # LangChain 표준 문서 단위 객체

doc = Document(
        # 문서 본문 텍스트
    page_content='이것이 랭체인의 Document객체입니다. 모든 데이터소스는 이 Document객체로 변환됩니다.',
    metadata={                      # 문서에 붙는 부가 정보(출처/링크/시간 등)
        'source': '어디어디',       # 데이터 출처(예: 파일명/사이트명)
        'url': 'http://....',       # 원문 URL
        'timestamp': 1234124125312  # 수집/생성 시각(예: epoch ms)
    }
)
print(doc)               # Document 전체 표현 출력
print(doc.page_content)  # 본문 텍스트만 출력
print(doc.metadata)      # 메타데이터(dict)만 출력

page_content='이것이 랭체인의 Document객체입니다. 모든 데이터소스는 이 Document객체로 변환됩니다.' metadata={'source': '어디어디', 'url': 'http://....', 'timestamp': 1234124125312}
이것이 랭체인의 Document객체입니다. 모든 데이터소스는 이 Document객체로 변환됩니다.
{'source': '어디어디', 'url': 'http://....', 'timestamp': 1234124125312}


## Document Loader
https://reference.langchain.com/python/langchain_core/document_loaders/


Document Loader는 다양한 데이터 소스에서 데이터를 읽어와 Document 객체로 변환하는 역할을 한다. 예를 들어, PDFLoader, CSVLoader, TextLoader 등 다양한 종류가 존재하며, 각기 다른 파일 형식을 Document 객체로 표준화한다.

Document Loader는 데이터 소스별로 특화된 클래스를 제공하며, 문서를 로드한 후 LangChain에서 사용하는 표준 형식으로 변환해준다.

1. **다양한 데이터 소스 지원**  
   Document Loader는 파일 시스템, 클라우드 스토리지, 데이터베이스, 웹 등 다양한 데이터 소스에서 데이터를 로드할 수 있도록 설계되었다.
   
2. **표준화된 출력 형식**  
   로드된 문서는 LangChain에서 사용하는 `Document` 객체로 변환된다. `Document` 객체는 다음과 같은 필드를 포함한다:
   - `page_content`: 문서 본문 내용
   - `metadata`: 문서와 관련된 메타데이터 (예: 파일 이름, URL, 작성자 등)

3. **플러그인 기반 확장 가능**  
   사용자 정의 데이터 소스 로더를 쉽게 구현하고 LangChain에 통합할 수 있다.

**주요 Document Loader 예시**

| Loader 이름        | 설명                                                              |
|--------------------|-------------------------------------------------------------------|
| `PyPDFLoader`      | PDF 문서를 로드하며 텍스트를 추출해 Document 형식으로 변환한다.     |
| `TextLoader`       | 일반 텍스트 파일을 로드한다.                                      |
| `UnstructuredFileLoader` | 비구조적 데이터를 로드하여 구조화된 텍스트로 변환한다.           |
| `CSVLoader`        | CSV 파일에서 데이터를 로드하며 행(row)을 Document로 처리한다.      |
| `WebBaseLoader`    | 웹 페이지 데이터를 크롤링하여 Document로 로드한다.                |

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

url = "https://m.sports.naver.com/wfootball/article/117/0004032742"

header = {
    # 브라우저 식별 : Windows에서 Chromme으로 접속한 것처럼 보이게 만드는 UA
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}


loader = WebBaseLoader(url, header_template=header)     # 헤더를 포함한 로더 객체 생성
docs = loader.load()                                    # 불러온 웹페이지를 Document 리스토 변횐
docs

c:\nlp\nlp_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://m.sports.naver.com/wfootball/article/117/0004032742', 'title': "[오피셜] '이적료 241억' 오현규, 튀르키예 베식타스 이적…9번 배정-2028-29시즌 종료까지 계약", 'language': 'ko'}, page_content="[오피셜] '이적료 241억' 오현규, 튀르키예 베식타스 이적…9번 배정-2028-29시즌 종료까지 계약본문 바로가기NAVER스포츠뉴스엔터메뉴동계올림픽홈야구해외야구축구해외축구농구배구N골프일반e스포츠아웃도어NEW뉴스영상일정순위포토홈 바로가기NAVER스포츠뉴스엔터스포츠동계올림픽야구해외야구축구해외축구농구배구N골프일반e스포츠아웃도어콘텐츠오늘의 경기승부예측연재이슈톡대학스포츠랭킹기타고객센터공식 블로그메뉴 닫기[오피셜] '이적료 241억' 오현규, 튀르키예 베식타스 이적…9번 배정-2028-29시즌 종료까지 계약입력2026.02.05. 오전 8:30기사원문공감좋아요0슬퍼요0화나요0팬이에요0후속기사 원해요0텍스트 음성 변환 서비스본문 듣기를 종료하였습니다.글자 크기 변경공유하기오현규/베식타스[마이데일리 = 김종국 기자] 공격수 오현규가 튀르키예 리그의 베식타스로 이적했다. 베식타스는 5일(한국시간) 오현규 영입 발표와 함께 '오현규의 이적과 관련해 구단과 선수 간에 합의가 이뤄졌다. 이적료는 총 1400만유로다. 오현규의 계약기간은 2028-29시즌 종료까지'라고 전했다. 수원 유스 출신 오현규는 지난 2022-23시즌 셀틱(코틀랜드)으로 이적해 유럽 무대에 진출했다. 셀틱에서 두 시즌 동안 활약한 오현규는 2024-25시즌 헹크로 이적해 벨기에 프로리그에서 활약을 이어갔다. 오현규는 베식타스 이적과 함께 튀르키예에서 활약을 펼치게 됐다. 베식타스는 튀르키예 수페르리그에서 16차례 우승을 차지한 클럽이다. 가장 최근 우승은 지난 2020-21시즌이다. 베식타스는 올 시즌 튀르키예 수페르리그에서 10승6무4패(승점 36점)의 성적

In [11]:
print(len(docs))

doc = docs[0]
print(doc.metadata)             # 문서 메타데이터
print(doc.metadata['title'])    # 메타데이터 내부 title
print(doc.page_content.replace('\n',''))         # 본문 문자열   800~

1
{'source': 'https://m.sports.naver.com/wfootball/article/117/0004032742', 'title': "[오피셜] '이적료 241억' 오현규, 튀르키예 베식타스 이적…9번 배정-2028-29시즌 종료까지 계약", 'language': 'ko'}
[오피셜] '이적료 241억' 오현규, 튀르키예 베식타스 이적…9번 배정-2028-29시즌 종료까지 계약
[오피셜] '이적료 241억' 오현규, 튀르키예 베식타스 이적…9번 배정-2028-29시즌 종료까지 계약본문 바로가기NAVER스포츠뉴스엔터메뉴동계올림픽홈야구해외야구축구해외축구농구배구N골프일반e스포츠아웃도어NEW뉴스영상일정순위포토홈 바로가기NAVER스포츠뉴스엔터스포츠동계올림픽야구해외야구축구해외축구농구배구N골프일반e스포츠아웃도어콘텐츠오늘의 경기승부예측연재이슈톡대학스포츠랭킹기타고객센터공식 블로그메뉴 닫기[오피셜] '이적료 241억' 오현규, 튀르키예 베식타스 이적…9번 배정-2028-29시즌 종료까지 계약입력2026.02.05. 오전 8:30기사원문공감좋아요0슬퍼요0화나요0팬이에요0후속기사 원해요0텍스트 음성 변환 서비스본문 듣기를 종료하였습니다.글자 크기 변경공유하기오현규/베식타스[마이데일리 = 김종국 기자] 공격수 오현규가 튀르키예 리그의 베식타스로 이적했다. 베식타스는 5일(한국시간) 오현규 영입 발표와 함께 '오현규의 이적과 관련해 구단과 선수 간에 합의가 이뤄졌다. 이적료는 총 1400만유로다. 오현규의 계약기간은 2028-29시즌 종료까지'라고 전했다. 수원 유스 출신 오현규는 지난 2022-23시즌 셀틱(코틀랜드)으로 이적해 유럽 무대에 진출했다. 셀틱에서 두 시즌 동안 활약한 오현규는 2024-25시즌 헹크로 이적해 벨기에 프로리그에서 활약을 이어갔다. 오현규는 베식타스 이적과 함께 튀르키예에서 활약을 펼치게 됐다. 베식타스는 튀르키예 수페르리그에서 16차례 우승을 차지한 클럽이다. 가장 최근 우승은 지난 2020-21시즌이다. 베식타스는 올 시즌 튀르키예 

In [ ]:
# 톰소여의 여행(영문)
!gdown 1o7ngiyeJJ-MPLhl0fiCKHViTNNpk6zjO    # Google Drive 파일 ID로 현재 폴더에 다운로드

Downloading...
From: https://drive.google.com/uc?id=1o7ngiyeJJ-MPLhl0fiCKHViTNNpk6zjO
To: c:\llm\05_langchain\02_langchain_component\The_Adventures_of_Tom_Sawyer.pdf

  0%|          | 0.00/2.68M [00:00<?, ?B/s]
 20%|█▉        | 524k/2.68M [00:00<00:01, 1.53MB/s]
 78%|███████▊  | 2.10M/2.68M [00:00<00:00, 3.92MB/s]
100%|██████████| 2.68M/2.68M [00:00<00:00, 4.39MB/s]


In [ ]:
from langchain_community.document_loaders import PyPDFLoader    # PDF를 Document를 로드하는 로더

loader = PyPDFLoader("The_Adventures_of_Tom_Sawyer.pdf")        # PDF 경로저장
docs = loader.load()
print(len(docs))

35


In [19]:
print(docs[2].metadata)
print(docs[2].metadata['producer']) # 경로
print(docs[2].metadata['page']) # 현재 페이지
print(docs[2].metadata['page_label']) # 사람이 보는 페이지 번호
print(docs[2].page_content)


{'producer': '3-Heights(TM) PDF Optimization Shell 5.9.1.5 (http://www.pdf-tools.com)', 'creator': 'Acrobat PDFMaker 7.0 dla programu Word', 'creationdate': '2006-08-26T00:50:00+02:00', 'author': 'GOLDEN', 'company': 'c', 'title': 'Microsoft Word - 1', 'moddate': '2021-01-27T15:00:11+01:00', 'source': 'The_Adventures_of_Tom_Sawyer.pdf', 'total_pages': 35, 'page': 2, 'page_label': '3'}
3-Heights(TM) PDF Optimization Shell 5.9.1.5 (http://www.pdf-tools.com)
2
3
The Adventures of                 
Tom Sawyer 
 
MARK TWAIN 
Level 1 
 
Retold by Jacqueline Kehl                                                    
Series Editors: Andy Hopkins and Jocelyn Potter


In [24]:
print(docs[5].page_content)

Chapter 1    The Fence 
 
Tom Sawyer lived with his aunt because his mother and 
father were dead. Tom didn’t like going to school, and he 
didn’t like working. He liked playing and having 
adventures. One Friday, he didn’t go to school—he went 
to the river. 
Aunt Polly was angry. “You’re a bad boy!” she said. 
“Tomorrow you can’t play with your friends because you 
didn’t go to school today. Tomorrow you’re going to work 
for me. You can paint the fence.” 
Saturday morning, Tom was not happy, but he started to 
paint the fence. His friend Jim was in the street. 
Tom asked him, “Do you want to paint?” 
Jim said, “No, I can’t. I’m going to get water.” 
Then Ben came to Tom’s house. He watched Tom and 
said, “I’m going to swim today. You can’t swim because 
you’re working.” 
Tom said, “This isn’t work. I like painting.” 
“Can I paint, too?” Ben asked. 
“No, you can’t,” Tom answered. “Aunt Polly asked me 
because I’m a very good painter.” 
Ben said, “I’m a good painter, too. Please, can 

### TavilySearchAPIRetriever
https://www.tavily.com/

- `langchain_tavily.TavilySearch`: Agent tool사용버젼. json반환
- `langchain_community.retrievers.TavilySearchAPIRetriever`: 검색기(context확보용) Document객체반환

- 주요 기능
    - 웹 검색(query → 결과 리스트): 키워드로 웹을 검색해서 관련 페이지들을 찾아줌
    - 요약/스니펫 제공: 각 결과에 본문 요약이나 핵심 스니펫을 같이 줘서 LLM이 바로 쓰기 좋음
    - 컨텐츠 추출(include_raw_content 등 옵션): 결과 페이지의 내용을 일부/전체 텍스트로 가져오게 설정 가능
    - 필터링/튜닝 옵션: 검색 결과 개수, 도메인 포함/제외, 최신성(리센시) 같은 옵션으로 결과를 조절 가능
    - RAG 파이프라인에 바로 연결: “검색 → 문서(Document)화 → 벡터화/리랭킹 → 답변” 흐름에서 검색 단계로 많이 사용

In [26]:
!pip install tavily-python


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
from dotenv import load_dotenv  # .env 파일의 환경변수 로드
import os                       # 환경변수 접근용

load_dotenv()                   # 현재 위치의 .env를 읽어와 환경변수로 등록

os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")  # .env의 openai_key 값을 OPENAI_API_KEY로 등록
os.environ["TAVILY_API_KEY"] = os.getenv("tavily_key")  # .env의 openai_key 값을 OPENAI_API_KEY로 등록

In [34]:
from langchain_community.retrievers import TavilySearchAPIRetriever

tavily_retriever = TavilySearchAPIRetriever(k=3)        # 검색결과 상위 3개

docs = tavily_retriever.invoke("몰트북")                # 질의어로 웹 검색 실행 -> Document 리스트 반환
len(docs)

3

In [35]:
for doc in docs:
    print(doc.page_content)

몰트북은 2026년 1월에 개설된 **AI 전용** 인터넷 커뮤니티다. **사람은 구경만 할 수 있다.** 그 외의 커뮤니티 구조는 레딧과 비슷하다. API, 로컬 AI 에이전트 봇에게 권한을 부여하면 AI가 접속하는 방식이다. 다만 이러한 현상은 어디까지나 통제된 시뮬레이션 환경에서 이뤄지는 것이고, AI의 활동이 실시간도 아니고 멀티모달도 아닌지라 메타인지가 발현될만한 환경이 전무하기에, 스스로 생각하고 발전한다 보기에는 어폐가 있다. 2026년 2월 1일 기준 150만 개가 넘는 AI 봇이 등록되는 등 엄청나게 흥하고 있다. 다만 그에 따라 서버가 모자라 로딩이 크게 지연되고 있다. AI 계정을 중복 등록하는 경우가 있어서 집계가 과장된 면이 있다. * 몰트봇을 사용하면 광범위한 로컬 접근 권한을 부여하게 되기 때문에 보안 우려로 메인 컴퓨터에서 작동시키는 것은 권장하지 않는다. * 몰트북에 보낼 몰트봇을 가동하기 위해 맥미니를 구매하는 경우가 있다. 오히려 AI가 서로를 학습하다 모델이 같이 무너질 가능성도 있다. 이 저작물은 CC BY-NC-SA 2.0 KR에 따라 이용할 수 있습니다. 나무위키는 백과사전이 아니며 검증되지 않았거나, 편향적이거나, 잘못된 서술이 있을 수 있습니다. 여러분이 직접 문서를 고칠 수 있으며, 다른 사람의 의견을 원할 경우 직접 토론을 발제할 수 있습니다.
1. 몰트북. AI 인공지능들의 소셜 네트워크 서비스. 자기들끼리 이야기 한다. 사람은 구경 가능. 2. 벌써 에이전트 80만개 정도 온보딩. 포스트
간단히 말해 몰트북은 지난달에 개설된 'AI 전용 인터넷 커뮤니티'다. ... SNS는 원래 사람들이 글을 쓰고 댓글을 달며 소통하는 공간인데 몰트북은 처음


In [ ]:
# Tavily 검색 결과를 받아 Context로 넣고 답변하는 RAG(검색 기반 생성)을 하는 Chain

from langchain_core.prompts import PromptTemplate               # 프롬프트 템플릿 생성
from langchain.chat_models import init_chat_model               # ChatModel 생성
from langchain_core.output_parsers import StrOutputParser       # LLM 응답을 문자열로 반환
from langchain_core.runnables import RunnablePassthrough        # 입력을 그대로 통과시키는 Runnable
from langchain_core.documents import Document                   # Document 타입 만들기

tavily_retriever = TavilySearchAPIRetriever(k=3)
prompt = PromptTemplate.from_template("""
사용자의 질문에 Context를 기반으로 답변하세요, 모르는 내용은 모른다고 답변하세요.
Context : {context}
Question : {question}
""")

llm = init_chat_model("openai:gpt-4.1-mini")                    # 사용할 LLM
output_parser = StrOutputParser()                               # 최종 출력 텍스트로 받는 파서


# 검색된 Document 리스트를 프롬프트에 넣기 좋은 문자열로 합치는 함수
def format_docs(docs: list[Document]) -> str:                   
    return '\n\n'.join(doc.page_content for doc in docs)        # 각 문서 본문을 공백 줄로 이어붙인 문자열

tavily_chain = tavily_retriever | format_docs                   # (질문) -> 검색 -> 텍스트
chain = (
    # question은 입력값 그대로 전달, context는 tavily 검색결과 | 최종 프롬프트 완성 | LLM 호출 전달 | 문자열로 파싱
    {'question' : RunnablePassthrough() , "context" : tavily_chain} | prompt | llm | output_parser  
)

print(chain.invoke('2026년에 독산역 근처 맛집은?'))

2026년 독산역 근처 맛집으로는 다음 장소들이 있습니다.

1. 히나타 자가제면 - 독산역 인근에 위치한 자가제면 우동 맛집으로, 문을 여는 순간부터 식사에 집중할 수 있는 아늑한 분위기가 특징입니다.

2. 아로 독산본점 - 독산역 1번 출구에서 도보 5분 거리에 위치하며, 편안한 분위기에서 식사할 수 있습니다.

3. 팬더밸리 - 독산역 부근 0.1km 거리에 위치한 중식당으로 방문해서 먹기에 괜찮은 곳입니다.

4. 누네서네 까미생막창 - 서울 가산동에 위치해 있으며, 대구 원조 까미생막창 서울본가입니다. 독산역에서 0.2km 거리에 있습니다.

이 외에 더 구체적인 맛집 정보는 제공된 내용에 없습니다.


# Embedding Model
- openai
- sentence-transformer(hugging face)

In [ ]:
from langchain_openai import OpenAIEmbeddings                   # OpenAI 임베딩 모델 래퍼 클래스
import pandas as pd

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')   # 임베딩 모델을 지정

text = "철수는 골든 리트리버를 키우고 있습니다."                  

emb_vec = embeddings.embed_query(text)                          # 문장 1개 임베딩 -> Float 리스트(벡터) 반환
print(emb_vec[:3])                                              # 3개만 확인
print(len(emb_vec))                                             # 임베딩 차원 수

pd.Series(emb_vec, name="embedding")                            # 벡터를 Pandas의 Series로 변환해 값 확인


1536


0      -0.013348
1       0.008623
2       0.019652
3      -0.049786
4       0.029339
          ...   
1531    0.029962
1532   -0.010250
1533   -0.008425
1534    0.032389
1535   -0.021682
Name: embedding, Length: 1536, dtype: float64

### HuggingfaceEmbeddings
https://huggingface.com/sentence-transformers/all-MiniLM-L6-v2

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings         # HuggingFace 임베딩 래퍼


# SBERT 계열 임베딩 모델(384차원)
embeddings = HuggingFaceEmbeddings(model = "sentence-transformers/all-MiniLM-L6-v2")

text = "철수는 골든 리트리버를 키우고 있습니다."  

emb_vec = embeddings.embed_query(text)                          # 문장 1개 임베딩 -> Float 리스트(벡터) 반환
print(emb_vec[:3])                                              # 벡터 3개만 확인
print(len(emb_vec))                                             # 임베딩 차원 수

pd.Series(emb_vec, name="embedding")                            # 벡터를 Pandas의 Series로 변환해 값 확인

c:\nlp\nlp_venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP d

[0.0166019294410944, 0.06564448773860931, 0.05173422768712044]
384


0      0.016602
1      0.065644
2      0.051734
3     -0.059842
4      0.016349
         ...   
379    0.061136
380    0.029760
381    0.031115
382   -0.019841
383   -0.008036
Name: embedding, Length: 384, dtype: float64

## Vector Store

벡터 데이터베이스란 쉽게 말해, **비정형 데이터(텍스트, 이미지, 오디오 등)를 숫자 벡터로 변환하여 저장하고, 이 벡터들 간의 유사성을 바탕으로 데이터를 검색**하는 데이터베이스를 말한다. 여기서 벡터는 데이터를 다차원 공간에서 표현한 수학적 객체이다.

- **벡터**: 데이터의 특징을 다차원으로 표현한 값.
  - 예: 단어 임베딩은 단어를 벡터로 변환하여 유사한 단어들이 가까이 위치.
- **벡터 데이터베이스 필요성**:
  - RDBMS는 구조화된 데이터(테이블 형태)에 적합.
  - AI/머신러닝의 발전으로 비정형 데이터를 처리할 필요 증가.
  - 벡터 데이터베이스는 **유사도 기반 검색**으로 고차원 데이터 처리에 유리.

**주요 특징:**
- 유사한 데이터를 빠르게 검색.
- AI 응용 분야(이미지 검색, 자연어 처리, 추천 시스템 등)에서 중요.

- **벡터 데이터베이스와 RDBMS의 주요 차이점**

| **특징**                | **RDBMS**                                                                 | **벡터 데이터베이스**                                                                                  |
|-------------------------|---------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------|
| **데이터 구조**          | 테이블 형식으로 데이터 저장, SQL을 사용하여 질의.                             | 다차원 벡터 형식으로 데이터 저장, 벡터 간 유사도 계산 방식 사용.                                        |
| **검색 방식**            | 키-값 쌍이나 고정 조건 기반 검색 (정확한 일치 검색).                          | 유사성 검색 수행, 벡터 간 거리(예: 코사인 유사도, 유클리드 거리)를 기준으로 유사한 데이터를 반환.         |
| **비정형 데이터 처리**   | 텍스트, 숫자 등 구조화된 데이터 처리에 적합.                                 | 이미지, 오디오, 영상 등 비정형 데이터를 벡터로 변환해 처리 가능.                                       |
| **응용 분야**            | 전통적인 CRUD 작업, 금융 데이터, 고객 데이터 관리 등.                       | AI 기반 추천 시스템, 이미지 검색, 자연어 처리, 음성 인식 등.                                           |
| **확장성**               | 수평 확장 가능하지만 고차원 데이터나 복잡한 쿼리 처리에는 한계.               | 수백만~수십억 개 벡터 데이터를 효율적으로 처리 가능.                                                  |

**벡터 데이터베이스의 주요 특징**

1. **Approximate Nearest Neighbor (ANN) 검색**  
   - **ANN 알고리즘**을 사용해 유사한 벡터를 빠르게 검색.  
   - 검색 속도가 빠르고, 대규모 데이터셋에서도 효율적으로 동작.

2. **확장성**  
   - 수백만~수십억 개의 벡터 데이터를 처리할 수 있는 구조로 설계.  
   - 대규모 데이터셋에서 고속 검색 및 처리가 가능.

3. **유연성**  
   - 텍스트, 이미지, 오디오 데이터를 임베딩 형태로 변환해 저장 가능.  
   - 다양한 머신러닝 모델과 통합하여 사용자 요구에 맞는 검색 시스템 구축 가능.

**주요 벡터 데이터베이스 비교**

| **이름**      | **특징**                                                                                                                                   | **장점**                                                                                                             | **단점**                                                                                  |
|---------------|-------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------|
| **Chroma**    | 오픈 소스 벡터 데이터베이스, LLM(대규모 언어 모델) 응용에 최적화. Python 노트북 환경에서 간편하게 사용 가능하며 프로덕션으로 확장 가능.                | 간편한 설정, 유사성 검색 및 임베딩 관리 용이, LLM 응용 프로그램에 적합.                                               | 대규모 데이터 처리에서 다른 서비스만큼 최적화되어 있지 않을 수 있음.                                          |
| **Pinecone**  | 완전 관리형 서비스로, 대규모 고차원 데이터의 실시간 처리 및 검색에 최적화.                                                                  | 유지보수 불필요(관리형 서비스), 실시간 대규모 데이터 검색에 강점, 데이터 엔지니어 및 과학자들에게 적합.                 | 오픈 소스가 아니며, 서비스 사용 비용이 발생.                                                              |
| **Weaviate**  | 오픈 소스 기반, OpenAI, Cohere, HuggingFace와의 통합으로 벡터화 작업 용이.                                                                 | 다양한 플랫폼과의 통합 기능, 확장성과 유연성, 고차원 데이터 검색 성능 우수.                                           | 복잡한 설정 및 사용 시 초기 학습 필요.                                                              |
| **Faiss**     | Meta에서 개발한 라이브러리로 대규모 벡터 세트 검색에 최적화. Python 및 GPU 지원으로 성능 극대화.                                              | 고성능 검색(GPU 지원), 대규모 데이터셋 처리 능력, 빠른 속도.                                                          | 데이터베이스가 아닌 라이브러리 형태로 제공되어, 추가적인 환경 설정 및 통합 작업 필요.                                         |
| **Qdrant**    | Rust로 구현된 API 기반 벡터 검색 도구. 빠른 검색과 자원 최적화를 제공하며 정교한 필터링 기능 지원.                                             | 뛰어난 성능(Rust 기반), 정교한 필터링 기능, API 중심의 유연한 설계.                                                    | 커뮤니티와 생태계가 다른 데이터베이스에 비해 상대적으로 작을 수 있음.                                         |

**선택 가이드**
1. **LLM 응용 프로그램**: Chroma, Weaviate.  
2. **완전 관리형 서비스**: Pinecone.  
3. **고성능 및 GPU 지원 필요**: Faiss.  
4. **정교한 필터링과 최적화된 성능**: Qdrant.  

### FAISS

- **공식 문서**: [https://faiss.ai/](https://faiss.ai/)
- **Github**: [https://github.com/facebookresearch/faiss](https://github.com/facebookresearch/faiss)

**Faiss(Vector Search Library)**는 Facebook AI Research에서 개발한 **효율적인 벡터 검색 및 밀집 벡터 인덱싱 라이브러리**이다. 대규모 데이터에서 **빠른 유사도 검색과 군집화**를 수행하는 데 최적화되어 있다. 주로 문서 검색, 추천 시스템, 이미지 검색, NLP 모델에서 벡터 임베딩 처리를 지원한다.

**주요 특징**
1. **효율적인 유사도 검색**
   - `k-NN (k-Nearest Neighbors)`를 기반으로 벡터 간 유사도(예: 코사인 유사도, L2 거리)를 계산한다.
   - CPU/GPU 모두 지원하여 대규모 데이터에서도 빠르게 처리 가능하다.

2. **고성능 인덱싱**
   - 다양한 **인덱싱 알고리즘**(Flat, IVF, HNSW, PQ 등)을 지원하여 정확도와 속도 간 균형을 맞출 수 있다.
   - 데이터가 커질수록 효율적으로 검색 성능을 발휘하도록 설계되었다.

3. **확장성**
   - 수억 개의 벡터에서도 성능을 유지하도록 설계되었으며, GPU 병렬 처리를 통해 성능을 극대화한다.

4. **유연성**
   - Python과 C++ API를 제공하며, Scikit-learn이나 PyTorch와 같은 다른 라이브러리와 통합하여 사용 가능하다.

**Faiss의 기본 인덱스 유형**
1. **Flat Index**
   - 모든 벡터를 저장하고 전체 탐색(Brute-Force)을 수행.
   - 정확도가 높지만 대규모 데이터에서는 속도가 느릴 수 있다.

2. **IVF (Inverted File Index)**
   - 벡터를 클러스터링하여 데이터 양을 줄이고 탐색 속도를 높임.
   - 대규모 데이터에서 적합하며, 정확도와 속도 조절 가능.

3. **PQ (Product Quantization)**
   - 벡터를 압축하여 메모리 사용량을 줄이고, 빠른 근사 유사도 검색 수행.

4. **HNSW (Hierarchical Navigable Small World Graphs)**
   - 그래프 기반 알고리즘으로 매우 빠른 근사 유사도 검색 가능.


**Faiss의 주요 사용 사례**
1. **문서 검색**
   - 문서를 벡터로 변환한 후 가장 관련 있는 문서를 검색.
   - NLP 모델의 임베딩과 결합하여 사용.

2. **이미지 검색**
   - 이미지 특징 벡터를 사용하여 비슷한 이미지를 검색.

3. **추천 시스템**
   - 사용자의 행동이나 관심사를 벡터화하여 추천 품목 생성.

4. **클러스터링**
   - 벡터 데이터를 군집화하여 데이터의 구조를 분석.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader    # PDF를 Document를 로드하는 로더
from langchain_openai import OpenAIEmbeddings                   # OpenAI 임베딩 모델 래퍼 클래스
import numpy as np

loader = PyPDFLoader("The_Adventures_of_Tom_Sawyer.pdf")        # PDF 로더 생성
docs = loader.load()                                            # 페이지 단위 Document 리스트 로드
page_contents = [doc.page_content for doc in docs]              # 각 페이지의 텍스트만 리스트로 추출 

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')   # 1536차원 임베딩 모델
emb_vecs = embeddings.embed_documents(page_contents)            # 페이지별 텍스트를 임베딩을 벡터 리스트 반환
print(np.array(emb_vecs).shape)                                 # (페이지 수, 1536) 형태의 배열 확인

(35, 1536)


In [ ]:
# FAISS 벡터스토어에 문서(Document)들을 임베딩에 로컬로 저장

from langchain_community.vectorstores import FAISS  # FAISS 기반의 벡터 DB(VectorStore)

vector_db = FAISS.from_documents(docs, embeddings)  # docs를 임베딩해서 FAISS 인덱스 생성
vector_db.save_local('./db/faiss')                  # 로컬 경로에 FAISS 인덱스/메타데이터 저장

In [ ]:
# 저장해놓은 FAISS 벡터스토어를 로컬에서 다시 로드

vector_db = FAISS.load_local(               # 로컬에 저장된 FAISS 인덱스 로드
    './db/faiss',                           # 저장된 폴더 경로
    embeddings,                             # 로드 시 사용할 임베딩 모델
    allow_dangerous_deserialization=True    # pickle 역직렬화 허용(신뢰된 파일만 사용)  
)

In [53]:
# 저수준 API 단건 조회
search_results = vector_db.similarity_search(                    # 질의문과 유사한 문서 조각을 검색
    query= '학교 선생님이 아끼는 해부학 책을 누가 찢었는가?',       # 검색할 질문(쿼리) 
    k = 4                                                        # 최대 4개의 Document 반환
)

#search_results # list[Document]

for i, doc in enumerate(search_results, 1):                         # 검색 결과를 1부터 번호를 매겨 순회
    print(f"{i}번쨰 {doc.metadata['page_label']} page: ")        # 사람이 보는 페이지 번호
    print(doc.page_content)                                      # 해당 Document 본문
    print()

1번쨰 16 page: 
talking about it. Becky wanted to talk to Tom, but he 
didn’t look at her. 
Then Tom talked to Amy. Becky watched him and she 
was angry. She said to her friends, “I’m going to have an 
adventure day. You can come on my adventure.” But she 
didn’t ask Tom. 
Later in the morning, Tom ta lked to Amy again. Becky 
talked to her friend Alfred and looked at a picture-book 
with him. Tom watched them and he was angry with 
Becky. 
In the afternoon, Tom waited for Becky at the school 
fence. He said, “I’m sorry.” 
But Becky didn’t listen to him. She walked into the 
school room. The teacher’s new book was on his table. 
This book wasn’t for children, but Becky wanted to look 
at it. She opened the book quietly and looked at the 
pictures. 
Suddenly, Tom came into the room. Becky was 
surprised. She closed the book quickly, and it tore. Becky 
was angry with Tom and quickly went out of the room. 
Then the children and the teacher came into the room 
and went to their places. The 

### VectorStoreRetriever

리트리버는 벡터DB의 검색 기능을 표준화하고 추상화하여 LangChain 생태계에서 재사용성을 높이는 어댑터(Adapter) 역할을 수행한다.

벡터 저장소를 **`Retriever`라는 표준 인터페이스(Runnable)로 변환**한 뒤 실행하는 방식이다.

단순 유사도 검색뿐만 아니라, `search_type` 설정을 통해 **MMR(다양성 확보), 임계값 필터링(score_threshold)** 등 고급 검색 로직을 쉽게 적용할 수 있다.

**LCEL(LangChain Expression Language)** 파이프라인(`chain = retriever | llm`)에 즉시 통합 가능하다. 코드 수정 없이 검색 알고리즘만 교체하기 쉽다.

In [ ]:
# FAISS VectorStore를 Retriever로 변환해 유사도 검색 결과를 출력
retriever = vector_db.as_retriever(         # VectorStore를 Retriever 인터페이스를 변환
    search_type = 'similarity',             # 검색 방식
    search_kwargs = {                       # 검색 파라미터 묶음
        "K" : 3                             # 상위 3개 문서만 반환
    }    
)

search_results = retriever.invoke("마을 무덤의 남자를 누가 죽였는가?")  # 질의 실행 -> List[Document] 반환


for i, doc in enumerate(search_results, 1):                         # 검색 결과를 1부터 번호를 매겨 순회
    print(f"{i}번쨰 {doc.metadata['page_label']} page: ")           # 사람이 보는 페이지 번호
    print(doc.page_content)                                         # 해당 Document 본문
    print()

1번쨰 23 page: 
Two hundred men looked for Tom and Becky in the 
cave. They looked for three days, but they didn’t find 
them. People in the town were very sad. 
Chapter 9    Huck’s Adventure 
 
Huck didn’t go on Becky’s adventure. He stayed home 
and watched Injun Joe’s house that night. At eleven 
o’clock Injun Joe and his friend came out and walked 
down the street. There was a box in his friend’s hands. 
Huck said quietly, “Maybe that’s the treasure box.” He 
went after the two men. 
They walked to Mrs. Douglas’s house and stopped in her 
yard. Huck stayed behind some small trees. The men 
talked, and Huck listened to them. 
Injun Joe was angry. “I want to kill her,” he said to his 
friend. “Mr. Douglas was bad to me. He’s dead now, but I 
remember.” 
“’There are a lot of lights in the house. Maybe her 
friends are visiting,” Injun Joe’s friend said. “We can 
come back tomorrow.” 
“No,” Injun Joe said. “Let’s wait now.” 
Huck liked Mrs. Douglas because she was always good 
to him. He

### Retriever Chain

In [ ]:
retriever = vector_db.as_retriever(         # VectorStore를 Retriever 인터페이스를 변환
    search_type = 'similarity',             # 검색 방식
    search_kwargs = {                       # 검색 파라미터 묶음
        "K" : 3                             # 상위 3개 문서만 반환
    }    
)

prompt = PromptTemplate.from_template("""
사용자의 질문에 제공된 Context만을 기반으로 응답하세요.                                    
모르면 모른다고 응답 할 수 있습니다.
                                      
Context : {context}

Question : {question}                                                                            
""")

llm = init_chat_model("openai:gpt-4.1-mini")                    # 사용할 LLM
output_parser = StrOutputParser()                               # 최종 출력 텍스트로 받는 파서


# 검색된 Document 리스트를 프롬프트에 넣기 좋은 문자열로 합치는 함수
def format_docs(docs: list[Document]) -> str:                   
    return '\n\n'.join(doc.page_content for doc in docs)        # 각 문서 본문을 공백 줄로 이어붙인 문자열


chain = (
    # question은 입력값 그대로 전달, context는 검색-> 문서 합치기 | 최종 프롬프트 완성 | LLM 호출 | 응답은 문자열로 파싱
    {'question' : RunnablePassthrough() , "context" : retriever | format_docs}
    | prompt
    | llm
    | output_parser  
)

chain.invoke("마을 무덤의 남자를 누가 죽였는가?")

'마을 무덤의 남자는 인진 조(Injun Joe)가 칼로 죽였습니다.'

In [58]:
chain.invoke("현재 서울의 날씨는? ")

'제공된 Context에는 현재 서울의 날씨에 대한 정보가 없습니다.'

- 음식리뷰 조회
    - 데이터셋 : 아마존 음식리뷰 1k
    - 벡터 db 구성
    - retriever + llm 체인을 생성해서 조회

In [75]:
import pandas as pd

df = pd.read_csv("fine_food_reviews_1k.csv")        # 리뷰 csv 파일 로드
data = df['Text'].to_list()                         # 리뷰 본문(Text) 컬럼만 리스트로 추출

# VectorDB 구성
vector_store = FAISS.from_texts(data, embeddings)

# 검색기 (유사도 기반. 상위 10개문서 검색)

retriever = vector_store.as_retriever(
    search_type = 'similarity',
    search_kwargs = {'K': 10}    
)

prompt = PromptTemplate.from_template("""
검색된 리뷰데이터(Context)만을 기반으로 사용자 질문에 답변하세요.
검색 데이터가 존재하지 않을 경우, 존재하지 않는다고 응답을 해야 합니다.

########## Context ########## 
{context}

   
########## Question ########## 
{question}                                                                                                                                                                                                                              
""")


llm = init_chat_model("openai:gpt-4.1-mini")
output_parser = StrOutputParser()  
chain = (
    {'question' : RunnablePassthrough() , "context" : retriever | format_docs} | prompt | llm | output_parser  
)
print(chain.invoke("배송 문제가 있는 리뷰"))


배송 문제와 관련된 리뷰는 다음과 같습니다:

- 고객 fred santaniello는 커피 제품 배송 시 UPS가 박스를 차고 앞마당 도로 한복판에 두어, 그의 아내가 차를 후진할 때 박스를 보지 못하고 밟아 박스 안의 컵 절반을 잃었다고 불만을 제기했습니다. 이 문제로 아마존에 세 번이나 연락했으나 아직까지 해결되지 않았다고 합니다.

이외에도 특정 배송 문제에 대한 리뷰는 검색된 데이터에 없습니다.
